## The goal here is to scan the (filtered) MSA with the depolymerase DB 

## 1. hmmcan the MSA
## 2. Scan the results
***

> <b>1. hmmscan command

In [ ]:
import os
import random
import subprocess
from multiprocessing import Pool
import time
import pandas as pd
# Generate a list of path of filtered MSA
path_proteins = "/home/conchae/ML_depolymerase/get_candidates/millard/scanned_MSA"

paths = [f"{path_proteins}/{msa}" for msa in os.listdir(f"{path_proteins}")]

def scan_depolymerase(path_query) :
    path_db = "/home/conchae/databases/depolymerase_building/DBsuite_depolymerase/depolymerase_db.suite"
    query = path_query.split("/")[-1].split(".MSA")[0]
    path_out = f"/home/conchae/ML_depolymerase/get_candidates/millard/DBsuite_depo/{query}"
    hhmscan_cmmd = f"hhsearch -i {path_query} -d  {path_db} -o {path_out}.suite.hhr -blasttab {path_out}.suite.tab"
    hhmscan_process = subprocess.Popen(hhmscan_cmmd, shell =True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)b} -o {path_out}.suite.hhr -blasttab {path_out}.suite.tab"
    hhmscan_process = subprocess.Popen(hhmscan_cmmd, shell =True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    scan_out, scan_err = hhmscan_process.communicate()

if __name__ == '__main__':
    with Pool(30) as p:
        p.map(scan_depolymerase, paths)
        
        
# *********************************************************** 
#!/bin/bash
    scan_out, scan_err = hhmscan_process.communicate()

if __name__ == '__main__':
    with Pool(30) as p:
        p.map(scan_depolymerase, paths)
        
        
# *********************************************************** 
#!/bin/bash
#BATCH --job-name=scan_ppt__
#SBATCH --qos=long
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=60 
#SBATCH --mem=200gb 
#SBATCH --time=10-00:00:00 
#SBATCH --output=scan_ppt__%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate HH-suite3

python /home/conchae/ML_depolymerase/scripts/dbdepo_scan.py

***
><b> 2. Scanning the outputs

In [ ]:
import os
#import random
#import subprocess
#from tqdm import tqdm
import time
import pandas as pd
#import pprint
import json 
#pp = pprint.PrettyPrinter(width = 150)
# Generate a list of path of filtered MSA 
path_proteins = "/home/conchae/ML_depolymerase/get_candidates/millard"
path_out = "/home/conchae/ML_depolymerase/get_candidates/millard/DBsuite_depo2"
path_tab = [f"{path_out}/{file}" for file in os.listdir(f"{path_out}") if file[-3:]=="tab"]

out_names = ["query","target","x1","hmm","mismatch","gapopen","query_start","query_stop","target_start","target_stop","evalue","score"]

def scan_tab(query_list) :
    scores = []
    dico_ipr = {}
    depolymerase_dico = {}
    for query in query_list:
        out_names = ["query","target","x1","hmm","mismatch","gapopen","query_start","query_stop","target_start","target_stop","evalue","score"]
        df_tab = pd.read_csv(f"{query}", sep="\t", names= out_names)
        dico_tab = df_tab.to_dict("records")
        for row in dico_tab :
            align_cols = int(row["query_stop"]) - int(row["query_start"])
            if align_cols > 30 and row["score"] > 20 :
                ipr = row["target"].split(".ready")[0]
                prot = row["query"]
                scores.append(row["score"])
                if prot not in depolymerase_dico :
                    positive_out = []
                    positive_out.append(f"{ipr}__{row['score']}")
                    depolymerase_dico[prot] = positive_out
                else :
                    depolymerase_dico[prot].append(f"{ipr}__{row['score']}")
                if ipr not in dico_ipr :
                    sco = []
                    sco.append(row["score"])
                    dico_ipr[ipr]=sco
                else :
                    dico_ipr[ipr].append(row["score"])
    dico_count = {ipr : len(dico_ipr[ipr]) for ipr in dico_ipr}
    return scores, dico_count, dico_ipr, depolymerase_dico

scores_out , dico_count_out, dico_ipr_out, depolymerase_dico_out = scan_tab(path_tab)

with open(f"{path_proteins}/dico_ipr_out.v2.json", "w") as outfile:
    json.dump(dico_ipr_out, outfile)
with open(f"{path_proteins}/dico_ipr_count.v2.json", "w") as outfile:
    json.dump(dico_count_out, outfile)    
with open(f"{path_proteins}/dbsuite_results.v2.json", "w") as outfile:
    json.dump(depolymerase_dico_out, outfile) 
    

    


> Check the annotation for each IPR ; get the unique() list

In [ ]:
import os
import time
import json 
from Bio import SeqIO
from tqdm import tqdm

#pp = pprint.PrettyPrinter(width = 150)
# Generate a list of path of filtered MSA 
path_proteins = "/home/conchae/ML_depolymerase/get_candidates/millard"
path_mill_db = "/home/conchae/databases/Millard_jan_2023/5Jan2023_vConTACT2_proteins.faa"

dico_res = json.load(open(f"{path_proteins}/dbsuite_results.v2.json"))
dico_count = json.load(open(f"{path_proteins}/dico_ipr_count.v2.json"))

proteins_annot = {record.id : " ".join(record.description.split(" ")[1:]) for record in SeqIO.parse(path_mill_db, "fasta") if record.id in dico_res}
proteins_seq = {record.id : record.seq for record in SeqIO.parse(path_mill_db, "fasta") if record.id in dico_res}

# Writting the fasta files :
with open(f"{path_proteins}/millard_depo.v2.fasta", "w") as outfile :
    for prot in tqdm(proteins_seq) :
        outfile.write(f">{prot} {proteins_annot[prot]}\n{proteins_seq[prot]}\n")

# Writting the results files :
for file in os.listdir(f"{path_proteins}/DBsuite_depo") :
    if file.split(".")[0] in proteins_annot :
        os.system(f"cp {path_proteins}/DBsuite_depo/{file} {path_proteins}/millard_dbsuite_results/{file}")
            
dico_annot = {}
for prot in dico_res :
    ipr = [ipr.split("__")[0] for ipr in dico_res[prot]]
    for index, ipr_entry in enumerate(ipr) :
        if ipr_entry not in dico_annot :
            ipr_annot = {proteins_annot[prot] : 1}
            dico_annot[ipr_entry] = ipr_annot
        elif proteins_annot[prot] not in dico_annot[ipr_entry]:
            a = {proteins_annot[prot] : 1}
            dico_annot[ipr_entry].update(a)
        else :
            dico_annot[ipr_entry][proteins_annot[prot]] += 1

for ipr in dico_annot :
    a = {"total_count" : dico_count[ipr]}
    dico_annot[ipr].update(a)

with open(f"{path_proteins}/results_annotation.v2.json", "w") as outfile:
    json.dump(dico_annot, outfile) 
with open(f"{path_proteins}/proteinID_annotation.v2.json", "w") as outfile:
    json.dump(proteins_annot, outfile) 

In [ ]:
rsync -avzhe ssh conchae@garnatxa.srv.cpd:/home/conchae/databases/depolymerase_building /media/concha-eloko/Linux/
rsync -avzhe ssh conchae@garnatxa.srv.cpd:/home/conchae/ML_depolymerase/get_candidates/millard/millard_depo.v1.fasta /media/concha-eloko/Linux/
rsync -avzhe ssh conchae@garnatxa.srv.cpd:/home/conchae/ML_depolymerase/get_candidates/millard/millard_dbsuite_results /media/concha-eloko/Linux/
rsync -avzhe ssh conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session /media/concha-eloko/Linux/


In [4]:
annotation_1 = {"IPR016283": {"endolysin": 891, "tail fiber protein": 41, "baseplate protein": 17, "hypothetical protein": 24, "structural protein": 6, "chitinase": 2, "tail protein": 4, "baseplate hub subunit and tail lysozyme": 11, "peptidase": 1, "virion structural protein": 10, "baseplate wedge subunit": 1, "total_count": 1008}, "IPR000726": {"endolysin": 725, "tail fiber protein": 41, "baseplate hub subunit and tail lysozyme": 24, "baseplate protein": 17, "hypothetical protein": 23, "structural protein": 6, "metallo-endopeptidase": 3, "chitinase": 2, "tail protein": 3, "peptidase": 1, "virion structural protein": 10, "Low affinity potassium transport system protein kup": 2, "baseplate wedge subunit": 1, "total_count": 858}, "IPR002152": {"internal virion protein with endolysin domain protein": 139, "unknown function": 99, "lytic tail protein": 100, "tRNA nucleotidyltransferase": 5, "tail protein with lysin activity": 14, "endolysin": 32, "transglycosylase": 64, "virion structural protein": 56, "tail fiber protein": 1, "hypothetical protein": 25, "DNA transfer protein": 13, "tail associated lysin": 1, "tail protein": 1, "tail length tape measure protein": 17, "putative transglycosylase IsaA": 3, "polynucleotide kinase": 1, "lytic tail fiber protein": 4, "Membrane-bound lytic murein transglycosylase B": 1, "total_count": 576}, "IPR005604": {"tail needle protein": 24, "tail protein": 261, "hypothetical protein": 245, "minor tail protein": 29, "tail fiber protein": 732, "tail spike protein": 177, "tail collar fiber protein": 35, "virion structural protein": 8, "terminase large subunit": 42, "minor head protein": 9, "haemolysin-type calcium binding protein": 8, "tail fiber assembly": 33, "head decoration": 64, "EPS depolymerase": 11, "glycoside hydrolase": 12, "long tail fiber protein distal subunit": 7, "colanic acid degradation": 13, "tail fiber protein proximal subunit": 1, "hydrolase": 2, "short tail fiber protein": 2, "baseplate wedge subunit": 1, "tail collar protein": 1, "total_count": 1717}, "IPR039473": {"long tail fiber protein proximal connector": 330, "tail protein": 10, "hypothetical protein": 29, "minor head protein": 13, "major tail protein": 3, "neck passage structure": 2, "tail assembly protein": 9, "tail fiber protein and host specificity": 9, "virion structural protein": 2, "glycerophosphoryl diester phosphodiesterase": 2, "tail fiber protein; host specificity": 2, "ParB-like partition protein": 1, "receptor binding tail protein": 2, "FabG-like 3-oxoacyl-(acyl-carrier-protein) reductase": 1, "SbcC-like subunit of palindrome specific endonuclease": 1, "total_count": 416}, "IPR014635": {"structural protein with Ig domain protein": 5, "DNA methyltransferase": 243, "Periplasmic alpha-amylase": 1, "tail protein": 2, "endolysin": 10, "major tail protein": 14, "FabG-like 3-oxoacyl-(acyl-carrier-protein) reductase": 2, "Oligo-1,6-glucosidase": 2, "head maturation protease": 24, "tail fiber protein": 2, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "hypothetical protein": 7, "DNA adenine methylase": 2, "Catabolic 3-dehydroquinate dehydratase": 1, "hydrolase": 1, "virion structural protein": 2, "Xylose isomerase": 1, "tail fiber protein; host specificity": 1, "Glycogen debranching enzyme": 4, "lysophospholipase L1-like esterase": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "total_count": 332}, "IPR013777": {"structural protein with Ig domain protein": 6, "DNA methyltransferase": 241, "Periplasmic alpha-amylase": 1, "tail protein": 6, "baseplate protein": 49, "minor tail protein": 4, "major tail protein": 21, "Oligo-1,6-glucosidase": 2, "hydrolase": 4, "head maturation protease": 29, "tail fiber protein": 5, "tail fiber protein; host specificity": 3, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "hypothetical protein": 10, "Catabolic 3-dehydroquinate dehydratase": 1, "virion structural protein": 2, "endolysin": 1, "Glycogen debranching enzyme": 4, "DNA adenine methylase": 1, "Tn3 family transposase TnAs3": 1, "total_count": 396}, "IPR006046": {"structural protein with Ig domain protein": 5, "DNA methyltransferase": 244, "Periplasmic alpha-amylase": 1, "major tail protein": 25, "tail protein": 5, "baseplate protein": 57, "hypothetical protein": 9, "Oligo-1,6-glucosidase": 2, "hydrolase": 4, "head maturation protease": 24, "tail fiber protein": 2, "Holliday junction resolvase": 1, "tail fiber protein; host specificity": 3, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "terminase large subunit": 1, "endolysin": 1, "Glycogen debranching enzyme": 4, "DNA adenine methylase": 1, "virion structural protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "total_count": 397}, "IPR001724": {"colanic acid degradation": 62, "endolysin": 7, "hypothetical protein": 269, "tail fiber protein": 1055, "EPS depolymerase": 41, "tail spike protein": 599, "virion structural protein": 23, "tail protein": 222, "long tail fiber protein distal subunit": 80, "major tail protein": 8, "major head protein": 25, "haemolysin-type calcium binding protein": 18, "baseplate wedge subunit": 15, "hinge connector of long tail fiber protein distal connector": 7, "pre-neck appendage protein": 24, "head protein": 32, "minor tail protein": 5, "portal protein": 3, "head scaffolding protein": 2, "baseplate wedge initiator": 5, "terminase large subunit": 9, "tail collar fiber protein": 2, "tail assembly protein": 1, "baseplate hub subunit and tail lysozyme": 18, "neck passage structure": 1, "Reverse transcriptase": 1, "tail fiber assembly": 2, "terminase": 1, "short tail fiber protein": 1, "DNA ejection": 2, "tail spike protein with colonic acid degradation activity": 8, "kinase": 1, "adaptor Ad4": 1, "total_count": 2550}, "IPR000933": {"tail protein": 11, "structural protein with Ig domain protein": 9, "major tail protein": 1, "hypothetical protein": 1, "total_count": 22}, "IPR016286": {"tail protein": 11, "structural protein with Ig domain protein": 9, "major tail protein": 1, "hypothetical protein": 1, "total_count": 22}, "IPR003790": {"tail protein": 14, "Pyridoxine 5'-phosphate synthase": 1, "Oligo-1,6-glucosidase": 2, "minor tail protein": 7, "major tail protein": 1, "pseudomurein endosiopeptidase": 1, "structural protein with Ig domain protein": 1, "hypothetical protein": 8, "lipase": 3, "Sulfoquinovosidase": 2, "hydrolase": 1, "Germination-specific N-acetylmuramoyl-L-alanine amidase": 1, "Beta-N-acetylhexosaminidase": 1, "SbcD-like subunit of palindrome specific endonuclease": 1, "Poly-beta-1,6-N-acetyl-D-glucosamine N-deacetylase": 1, "Glycogen debranching enzyme": 3, "tail fiber protein": 1, "Alpha-xylosidase": 1, "Hoc-like head decoration": 1, "total_count": 51}, "IPR000974": {"anti-repressor Ant": 59, "virion structural protein": 10, "hypothetical protein": 16, "tail length tape measure protein": 34, "putative transglycosylase IsaA": 22, "anti-repressor": 32, "tail fiber protein": 8, "transglycosylase": 16, "polynucleotide kinase": 1, "baseplate hub subunit and tail lysozyme": 1, "putative protein YqbO": 1, "total_count": 200}, "IPR005193": {"tail sheath": 135, "tail protein": 3, "levanase": 2, "queuine tRNA-ribosyltransferase": 2, "total_count": 142}, "IPR044914": {"tail fiber protein": 76, "tail spike protein": 75, "hypothetical protein": 64, "tail protein": 24, "minor tail protein": 1, "virion structural protein": 3, "baseplate wedge initiator": 1, "colanic acid degradation": 1, "total_count": 245}, "IPR024430": {"tail fiber protein": 44, "baseplate wedge subunit": 9, "tail spike protein": 62, "hypothetical protein": 20, "EPS depolymerase": 1, "tail protein": 17, "colanic acid degradation": 1, "total_count": 154}, "IPR039279": {"hypothetical protein": 361, "tail fiber protein": 105, "EPS depolymerase": 53, "tail protein": 170, "tail spike protein": 352, "minor tail protein": 27, "receptor binding tail protein": 7, "pre-neck appendage protein": 80, "colanic acid degradation": 35, "virion structural protein": 24, "tail collar protein": 4, "neck passage structure": 2, "pectin lyase activity": 1, "tRNA-His guanylyltransferase": 1, "head decoration": 1, "adaptor Ad4": 2, "total_count": 1225}, "IPR000514": {"hypothetical protein": 12, "hydrolase": 2, "minor tail protein": 1, "transcriptional regulator": 2, "6-phospho-beta-glucosidase BglB": 1, "major tail protein": 2, "total_count": 20}, "IPR001000": {"hypothetical protein": 7, "baseplate wedge tail fiber protein connector": 1, "6-phospho-beta-glucosidase BglB": 1, "Exoglucanase-2": 1, "tail spike protein": 1, "total_count": 11}, "IPR024745": {"hypothetical protein": 5, "5,10-methylenetetrahydrofolate reductase": 1, "total_count": 6}, "IPR003476": {"hypothetical protein": 14, "baseplate protein": 61, "tail protein": 8, "Oligo-1,6-glucosidase": 2, "hydrolase": 5, "L-ribulose-5-phosphate 3-epimerase UlaE": 2, "radical SAM domain-containing protein": 2, "Trehalose-6-phosphate hydrolase": 2, "DNA invertase": 1, "6-phospho-beta-glucosidase BglB": 1, "lipase": 3, "Peptidoglycan deacetylase": 1, "major tail protein": 1, "tail fiber protein": 1, "Sugar phosphatase YidA": 1, "total_count": 105}, "IPR011683": {"hypothetical protein": 8, "major tail protein": 9, "Oligo-1,6-glucosidase": 2, "hydrolase": 4, "RNA ligase and tail fiber protein attachment catalyst": 1, "head maturation protease": 1, "structural protein with Ig domain protein": 5, "Trehalose-6-phosphate hydrolase": 3, "tail protein": 4, "6-phospho-beta-glucosidase BglB": 1, "transglycosylase": 1, "Biotin synthase": 1, "total_count": 40}, "IPR005199": {"hypothetical protein": 6, "Guanine deaminase": 1, "total_count": 7}, "IPR001547": {"hypothetical protein": 12, "baseplate protein": 61, "RNA ligase": 28, "Oligo-1,6-glucosidase": 2, "hydrolase": 4, "Trehalose-6-phosphate hydrolase": 3, "tail protein": 4, "6-phospho-beta-glucosidase BglB": 1, "virion structural protein": 1, "endolysin": 1, "total_count": 117}, "IPR044846": {"hypothetical protein": 1, "Exoglucanase-2": 1, "total_count": 2}, "IPR039743": {"hypothetical protein": 9, "minor tail protein": 1, "major tail protein": 2, "total_count": 12}, "IPR039514": {"hypothetical protein": 4, "minor tail protein": 1, "major tail protein": 1, "total_count": 6}, "IPR002925": {"endolysin": 8, "Vitamin B12-dependent ribonucleoside-diphosphate reductase": 22, "esterase/lipase": 56, "hypothetical protein": 46, "Non-heme chloroperoxidase": 146, "Modification methylase RsrI": 4, "esterase": 27, "Haloacetate dehalogenase H-1": 2, "Arylesterase": 79, "Pimeloyl-[acyl-carrier protein] methyl ester esterase": 2, "Lipase 2": 2, "Modification methylase DpnIIB": 5, "peptidase": 5, "DNA methyltransferase": 3, "ribonucleotide reductase large subunit": 1, "RNA polymerase": 2, "Putative acetyl-hydrolase LipR": 1, "DNA polymerase exonuclease subunit": 8, "putative protein": 15, "ribonucleotide reductase": 5, "virion structural protein": 1, "Poly(3-hydroxyalkanoate) polymerase subunit PhaC": 4, "Aclacinomycin methylesterase RdmC": 2, "Modification methylase MboII": 1, "long tail fiber protein distal subunit": 1, "Lysophospholipase L2": 2, "DNA helicase": 1, "nucleotide-sugar epimerase": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 453}, "IPR024428": {"tail spike protein": 27, "tail fiber protein": 35, "hypothetical protein": 26, "major head protein": 21, "tail protein": 12, "EPS depolymerase": 2, "total_count": 123}, "IPR029456": {"hypothetical protein": 15, "tail fiber assembly": 3, "tail protein": 1, "major tail protein": 8, "Hoc-like head decoration": 1, "single strand DNA binding protein": 1, "total_count": 29}, "IPR000490": {"UvsX-like recombinase": 92, "Protein RecA": 12, "total_count": 104}, "IPR023720": {"DNA repair protein": 70, "UvsX-like recombinase": 13, "Protein RecA": 16, "recombinase": 1, "Cytoplasmic trehalase": 1, "hypothetical protein": 2, "total_count": 103}, "IPR015331": {"tail spike protein": 220, "colanic acid degradation": 2, "tail protein": 15, "hypothetical protein": 6, "tail fiber protein": 24, "haemolysin-type calcium binding protein": 3, "total_count": 270}, "IPR016828": {"tail protein": 10, "major tail protein": 62, "structural protein with Ig domain protein": 8, "tail fiber protein; host specificity": 1, "Inner membrane symporter YicJ": 1, "Extracellular exo-alpha-(1->5)-L-arabinofuranosidase": 1, "lipase": 1, "total_count": 84}, "IPR026283": {"tail protein": 12, "hydrolase": 4, "2-amino-3-ketobutyrate coenzyme A ligase": 3, "hypothetical protein": 4, "major tail protein": 1, "total_count": 24}, "IPR037110": {"tail protein": 4, "hypothetical protein": 1, "total_count": 5}, "IPR031330": {"tail protein": 8, "hydrolase": 4, "2-amino-3-ketobutyrate coenzyme A ligase": 3, "hypothetical protein": 3, "total_count": 18}, "IPR038964": {"tail protein": 13, "hypothetical protein": 1, "virion structural protein": 10, "total_count": 24}, "IPR006710": {"tail protein": 9, "Levanbiose-producing levanase": 1, "hypothetical protein": 2, "structural protein with Ig domain protein": 8, "major tail protein": 1, "Inner membrane symporter YicJ": 1, "Extracellular exo-alpha-(1->5)-L-arabinofuranosidase": 1, "levanase": 4, "lipase": 1, "total_count": 28}, "IPR016840": {"tail protein": 14, "Hoc-like head decoration": 1, "Levanbiose-producing levanase": 1, "hypothetical protein": 3, "structural protein with Ig domain protein": 5, "Inner membrane symporter YicJ": 1, "Extracellular exo-alpha-(1->5)-L-arabinofuranosidase": 1, "virion structural protein": 1, "levanase": 2, "lipase": 1, "major tail protein": 1, "total_count": 31}, "IPR043534": {"tail protein": 8, "hypothetical protein": 27, "head protein": 22, "Aldose 1-epimerase": 2, "Unsaturated 3S-rhamnoglycuronyl hydrolase": 1, "DNA invertase": 1, "virion structural protein": 2, "tail fiber protein": 1, "major tail protein": 2, "Hoc-like head decoration": 1, "total_count": 67}, "IPR003469": {"tail protein": 4, "Levanbiose-producing levanase": 1, "hypothetical protein": 1, "levanase": 2, "total_count": 8}, "IPR016007": {"tail protein": 4, "minor tail protein": 54, "virion structural protein": 15, "hypothetical protein": 9, "neck passage structure protein": 2, "Cytoplasmic trehalase": 1, "major tail protein": 2, "major head protein": 1, "total_count": 88}, "IPR001362": {"tail protein": 6, "Levanbiose-producing levanase": 1, "levanase": 2, "hypothetical protein": 1, "tail fiber protein": 1, "total_count": 11}, "IPR009860": {"hyaluronidase": 47, "hypothetical protein": 56, "tail fiber protein": 17, "hinge connector of long tail fiber protein distal connector": 1, "tail collar fiber protein": 1, "tail fiber protein proximal subunit": 3, "total_count": 125}, "IPR016289": {"SbcD-like subunit of palindrome specific endonuclease": 45, "minor tail protein": 65, "exonuclease": 8, "hydrolase": 4, "major tail protein": 13, "hypothetical protein": 12, "virion structural protein": 2, "endolysin": 1, "tail protein": 1, "total_count": 151}, "IPR034641": {"major tail protein": 75, "Ig domain containing protein": 1, "minor tail protein": 6, "tail protein": 16, "hypothetical protein": 7, "head maturation protease": 7, "virion structural protein": 2, "head closure Hc2": 3, "baseplate hub protein Tal": 4, "minor head protein": 2, "total_count": 123}, "IPR041624": {"major tail protein": 29, "hypothetical protein": 39, "virion structural protein": 11, "Exoglucanase-2": 5, "minor tail protein": 87, "Ig domain containing protein": 3, "S-(hydroxymethyl)mycothiol dehydrogenase": 1, "baseplate hub protein Tal": 4, "tail fiber protein": 1, "esterase/lipase": 1, "tail protein": 3, "endolysin": 2, "total_count": 186}, "IPR017069": {"Periplasmic alpha-amylase": 1, "tail protein": 2, "major tail protein": 9, "Oligo-1,6-glucosidase": 2, "structural protein with Ig domain protein": 13, "head maturation protease": 24, "tail fiber protein": 2, "nicotinamide phosphoribosyl transferase": 6, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "hypothetical protein": 10, "endolysin": 1, "Glycogen debranching enzyme": 4, "DNA adenine methylase": 1, "virion structural protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "Lipopolysaccharide core heptosyltransferase RfaQ": 1, "UvsX-like recombinase": 1, "total_count": 85}, "IPR013776": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "major tail protein": 19, "Oligo-1,6-glucosidase": 2, "Thymidylate synthase": 4, "head maturation protease": 25, "tail fiber protein": 2, "thymidylate synthase": 2, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "hypothetical protein": 7, "Thymidylate synthase 2": 1, "structural protein with Ig domain protein": 2, "tail fiber protein; host specificity": 1, "Glycogen debranching enzyme": 4, "virion structural protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "total_count": 81}, "IPR005323": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "major tail protein": 9, "hypothetical protein": 14, "Oligo-1,6-glucosidase": 2, "head maturation protease": 48, "tail fiber protein": 7, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 5, "virion structural protein": 3, "Glycogen debranching enzyme": 4, "structural protein with Ig domain protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 102}, "IPR011837": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "major tail protein": 9, "Oligo-1,6-glucosidase": 2, "head maturation protease": 40, "tail fiber protein": 5, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 4, "hypothetical protein": 9, "structural protein with Ig domain protein": 2, "Glycogen debranching enzyme": 4, "virion structural protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 85}, "IPR011840": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "major tail protein": 11, "hypothetical protein": 14, "Oligo-1,6-glucosidase": 2, "head maturation protease": 52, "tail fiber protein": 8, "tail fiber protein; host specificity": 1, "QueE-like radical SAM domain protein": 1, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "virion structural protein": 3, "Glycogen debranching enzyme": 4, "DnaB-like replicative helicase": 1, "structural protein with Ig domain protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 110}, "IPR011838": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "hypothetical protein": 13, "major tail protein": 7, "Oligo-1,6-glucosidase": 2, "head maturation protease": 48, "tail fiber protein": 6, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "virion structural protein": 2, "Glycogen debranching enzyme": 4, "structural protein with Ig domain protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 95}, "IPR013775": {"Periplasmic alpha-amylase": 1, "tail protein": 1, "major tail protein": 10, "Oligo-1,6-glucosidase": 2, "head maturation protease": 24, "tail fiber protein": 2, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "hypothetical protein": 3, "Glycogen debranching enzyme": 4, "virion structural protein": 1, "structural protein with Ig domain protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "total_count": 56}, "IPR022844": {"Periplasmic alpha-amylase": 1, "tail protein": 2, "major tail protein": 6, "Oligo-1,6-glucosidase": 2, "unknown function": 19, "head maturation protease": 40, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "hypothetical protein": 6, "tail fiber protein": 4, "Glycogen debranching enzyme": 4, "virion structural protein": 1, "structural protein with Ig domain protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "total_count": 93}, "IPR011839": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "major tail protein": 6, "hypothetical protein": 10, "Oligo-1,6-glucosidase": 2, "endolysin": 2, "head maturation protease": 48, "tail fiber protein": 7, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "virion structural protein": 4, "Glycogen debranching enzyme": 4, "structural protein with Ig domain protein": 1, "Tn3 family transposase TnAs3": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 95}, "IPR032792": {"Periplasmic alpha-amylase": 1, "Oligo-1,6-glucosidase": 2, "head maturation protease": 24, "tail fiber protein": 1, "DNA polymerase exonuclease subunit": 14, "Trehalose-6-phosphate hydrolase": 3, "major tail protein": 1, "hypothetical protein": 2, "Glycogen debranching enzyme": 2, "virion structural protein": 1, "structural protein with Ig domain protein": 1, "total_count": 52}, "IPR002252": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "Oligo-1,6-glucosidase": 2, "Tn3 family transposase TnAs1": 2, "hypothetical protein": 3, "Trehalose-6-phosphate hydrolase": 3, "Sulfoquinovosidase": 2, "Germination-specific N-acetylmuramoyl-L-alanine amidase": 1, "tail spike protein": 2, "Biotin synthase": 1, "Lactose permease": 1, "Alpha-xylosidase": 1, "Tn3 family transposase TnAs3": 1, "total_count": 23}, "IPR019563": {"hypothetical protein": 7, "deoxynucleoside monophosphate kinase": 2, "4-hydroxy-3-methylbut-2-en-1-yl diphosphate synthase (flavodoxin)": 3, "radical SAM domain-containing protein": 2, "tail fiber protein": 2, "Trehalose-6-phosphate hydrolase": 2, "Glycerophosphodiester phosphodiesterase": 1, "Catabolic 3-dehydroquinate dehydratase": 1, "Purine nucleoside phosphorylase DeoD-type": 2, "3-dehydroquinate dehydratase": 1, "L-ribulose-5-phosphate 3-epimerase UlaE": 1, "total_count": 24}, "IPR037398": {"major tail protein": 5, "hypothetical protein": 6, "Hoc-like head decoration": 3, "tail protein": 4, "fibritin neck whisker": 10, "total_count": 28}, "IPR026071": {"hypothetical protein": 6, "4-sulfomuconolactone hydrolase": 2, "2-amino-3-ketobutyrate coenzyme A ligase": 3, "radical SAM domain-containing protein": 3, "homing endonuclease": 1, "total_count": 15}, "IPR012970": {"major tail protein": 42, "tail fiber protein; host specificity": 2, "hypothetical protein": 3, "structural protein with Ig domain protein": 6, "total_count": 53}, "IPR011395": {"tail protein": 7, "Beta-N-acetylhexosaminidase": 1, "hypothetical protein": 1, "total_count": 9}, "IPR011100": {"tail protein": 7, "total_count": 7}, "IPR006215": {"tail protein": 11, "tail spike protein": 3, "hypothetical protein": 1, "total_count": 15}, "IPR000743": {"tail protein": 3, "hypothetical protein": 5, "tail spike protein": 2, "pre-neck appendage protein": 1, "total_count": 11}, "IPR002241": {"tail protein": 5, "tail spike protein": 3, "Alpha-xylosidase": 1, "total_count": 9}, "IPR039513": {"hypothetical protein": 21, "transcriptional repressor": 10, "major tail protein": 1, "virion structural protein": 4, "Hoc-like head decoration": 1, "Holliday junction ATP-dependent DNA helicase RuvB": 1, "DNA helicase": 1, "tail fiber protein": 2, "structural protein with Ig domain protein": 2, "Ribosomal RNA small subunit methyltransferase E": 2, "Ferredoxin--NADP reductase": 1, "hydrolase": 1, "neck protein": 2, "Stage V sporulation protein K": 1, "tail spike protein": 1, "protease": 1, "distal tail protein": 1, "total_count": 53}, "IPR000400": {"baseplate hub subunit and tail lysozyme": 39, "hypothetical protein": 45, "endolysin": 5, "dehydrogenase": 1, "baseplate protein": 1, "total_count": 91}, "IPR011583": {"minor tail protein": 66, "protease": 1, "tail protein": 5, "hypothetical protein": 17, "endolysin": 1, "major tail protein": 3, "Dda-like helicase": 1, "total_count": 94}, "IPR000677": {"minor tail protein": 66, "major tail protein": 10, "protease": 1, "hydrolase": 5, "tail protein": 6, "hypothetical protein": 17, "Urease accessory protein UreG": 1, "endolysin": 1, "Cytolethal distending toxin subunit B": 1, "Dda-like helicase": 1, "tail spike protein": 1, "SbcD-like subunit of palindrome specific endonuclease": 1, "head maturation protease": 1, "total_count": 112}, "IPR038970": {"structural protein with Ig domain protein": 8, "major tail protein": 23, "hypothetical protein": 2, "tail fiber protein; host specificity": 1, "3-(3-hydroxy-phenyl)propionate/3-hydroxycinnamic acid hydroxylase": 1, "total_count": 35}, "IPR045032": {"tail protein": 8, "tail spike protein": 55, "pectate lyase": 3, "pre-neck appendage protein": 26, "major tail protein": 7, "hypothetical protein": 21, "hydrolase": 4, "virion structural protein": 1, "minor tail protein": 1, "total_count": 126}, "IPR018082": {"tail protein": 6, "tail spike protein": 34, "Hoc-like head decoration": 4, "pectate lyase": 3, "pre-neck appendage protein": 26, "tail fiber protein": 2, "major tail protein": 7, "hypothetical protein": 14, "hydrolase": 4, "tail assembly protein": 3, "virion structural protein": 4, "total_count": 107}, "IPR002022": {"tail protein": 6, "tail spike protein": 35, "pectate lyase": 3, "pre-neck appendage protein": 26, "major tail protein": 7, "hypothetical protein": 20, "hydrolase": 4, "virion structural protein": 1, "tail fiber protein": 1, "total_count": 103}, "IPR025975": {"peptidase": 8, "hypothetical protein": 63, "virion structural protein": 1, "endolysin": 1, "total_count": 73}, "IPR011496": {"peptidase": 61, "Sak4-like ssDNA annealing protein": 4, "Hydroxymethylglutaryl-CoA lyase YngG": 2, "hypothetical protein": 1, "total_count": 68}, "IPR008397": {"ribonucleoside diphosphate reductase small subunit": 2, "Ribonucleoside-diphosphate reductase 2 subunit beta": 1, "hypothetical protein": 6, "tail fiber protein": 2, "total_count": 11}, "IPR008929": {"ribonucleoside diphosphate reductase small subunit": 2, "hypothetical protein": 8, "Ribonucleoside-diphosphate reductase 2 subunit beta": 1, "total_count": 11}, "IPR016455": {"glucosyltransferase": 17, "hypothetical protein": 15, "total_count": 32}, "IPR016287": {"glucosyltransferase": 14, "hypothetical protein": 37, "Ig domain containing protein": 1, "major tail protein": 2, "tail protein": 1, "structural protein with Ig domain protein": 2, "virion structural protein": 1, "total_count": 58}, "IPR000757": {"glucosyltransferase": 14, "hypothetical protein": 14, "Ig domain containing protein": 1, "structural protein with Ig domain protein": 1, "total_count": 30}, "IPR016282": {"baseplate protein": 61, "major tail protein": 10, "hydrolase": 4, "Ribonucleoside-diphosphate reductase subunit alpha": 1, "hypothetical protein": 3, "tail protein": 6, "tail fiber protein": 2, "endolysin": 1, "total_count": 88}, "IPR000556": {"major tail protein": 17, "virion structural protein": 5, "Exoglucanase-2": 3, "tail protein": 31, "Ig domain containing protein": 1, "minor tail protein": 4, "hypothetical protein": 3, "tail assembly protein": 3, "endolysin": 1, "total_count": 68}, "IPR025706": {"structural protein with Ig domain protein": 1, "tail fiber protein; host specificity": 7, "Oligo-1,6-glucosidase": 2, "major tail protein": 1, "tail protein": 1, "Trehalose-6-phosphate hydrolase": 1, "total_count": 13}, "IPR024200": {"major tail protein": 13, "hypothetical protein": 1, "tail fiber protein; host specificity": 2, "Thymidylate synthase": 2, "endolysin": 1, "thymidylate synthase": 1, "total_count": 20}, "IPR003159": {"major tail protein": 20, "hypothetical protein": 3, "tail fiber protein; host specificity": 1, "structural protein with Ig domain protein": 2, "total_count": 26}, "IPR039174": {"major tail protein": 8, "hypothetical protein": 1, "tail fiber protein; host specificity": 2, "endolysin": 1, "total_count": 12}, "IPR014895": {"hypothetical protein": 10, "total_count": 10}, "IPR018155": {"hypothetical protein": 1, "minor tail protein with lysin activity": 1, "2OG-Fe(II) oxygenase": 1, "Methionine synthase": 1, "total_count": 4}, "IPR001329": {"hypothetical protein": 3, "2OG-Fe(II) oxygenase": 1, "Homoserine O-succinyltransferase": 1, "total_count": 5}, "IPR001088": {"UDP-glucose dehydrogenase": 11, "Cell division protein FtsA": 2, "hypothetical protein": 15, "N-acetyl-gamma-glutamyl-phosphate reductase": 1, "FabG-like 3-oxoacyl-(acyl-carrier-protein) reductase": 6, "UDP-glucose 6-dehydrogenase TuaD": 2, "3-hydroxyadipyl-CoA dehydrogenase": 2, "4-hydroxy-tetrahydrodipicolinate reductase": 2, "dehydrogenase": 5, "Lysine 6-dehydrogenase": 1, "Malate dehydrogenase": 1, "Pyrroline-5-carboxylate reductase": 2, "Cell division protein FtsZ": 2, "Sulfofructosephosphate aldolase": 1, "6-phospho-alpha-glucosidase": 1, "Carbamoyl-phosphate synthase small chain": 1, "Alpha-galactosidase": 1, "DNA gyrase subunit B": 3, "ribonucleotide reductase large subunit": 1, "Glycerol-3-phosphate dehydrogenase [NAD(P)+]": 3, "NADPH oxidoreductase": 1, "tryptophan halogenase": 1, "Flavin-dependent tryptophan halogenase RebH": 1, "Chromosome-partitioning ATPase Soj": 1, "Heme response regulator HssR": 1, "Fatty acid oxidation complex subunit alpha": 2, "Phosphoribosylformylglycinamidine synthase subunit PurQ": 1, "UDP-N-acetylglucosamine 4,6-dehydratase (inverting)": 1, "DNA topoisomerase II large subunit": 1, "UDP-glucose 6-dehydrogenase": 1, "metal-dependent hydrolase": 1, "total_count": 75}, "IPR002037": {"major tail protein": 5, "hypothetical protein": 2, "structural protein with Ig domain protein": 12, "total_count": 19}, "IPR023309": {"Exoglucanase-2": 3, "major tail protein": 1, "total_count": 4}, "IPR012878": {"minor tail protein": 1, "glycosyltransferase": 9, "hypothetical protein": 4, "total_count": 14}, "IPR016714": {"hypothetical protein": 8, "tail protein": 3, "virion structural protein": 1, "tail fiber assembly": 1, "total_count": 13}, "IPR041351": {"hypothetical protein": 9, "virion structural protein": 1, "total_count": 10}, "IPR041542": {"hypothetical protein": 4, "fibritin neck whisker": 10, "tail fiber protein": 1, "total_count": 15}, "IPR001286": {"hypothetical protein": 9, "major tail protein": 4, "tail fiber protein": 4, "tail spike protein": 1, "tail protein": 2, "total_count": 20}, "IPR006775": {"tail fiber protein": 2, "neck passage structure protein": 2, "endolysin": 1, "hypothetical protein": 3, "glycosyltransferase": 1, "total_count": 9}, "IPR014551": {"tail fiber protein": 2, "neck passage structure protein": 2, "endolysin": 1, "hypothetical protein": 2, "glycosyltransferase": 1, "total_count": 8}, "IPR005198": {"tail fiber protein": 2, "hypothetical protein": 8, "Sulfoquinovose isomerase": 2, "Unsaturated 3S-rhamnoglycuronyl hydrolase": 1, "tail protein": 2, "minor tail protein": 1, "total_count": 16}, "IPR035396": {"tail fiber protein": 2, "neck passage structure protein": 2, "hypothetical protein": 10, "minor tail protein": 1, "Cytoplasmic trehalase": 1, "major tail protein": 1, "glycosyltransferase": 1, "major head protein": 1, "total_count": 19}, "IPR015289": {"tail protein": 8, "hypothetical protein": 7, "virion structural protein": 13, "minor tail protein": 1, "tail fiber assembly": 1, "baseplate wedge tail fiber protein connector": 1, "total_count": 31}, "IPR001371": {"hypothetical protein": 5, "total_count": 5}, "IPR035669": {"hypothetical protein": 6, "esterase/lipase": 2, "ATPase": 4, "lipase": 3, "virion structural protein": 1, "total_count": 16}, "IPR021016": {"minor tail protein": 1, "total_count": 1}, "IPR008902": {"neck passage structure protein": 2, "hypothetical protein": 1, "major head protein": 1, "total_count": 4}, "IPR001439": {"hypothetical protein": 14, "2OG-Fe(II) oxygenase": 2, "acetyltransferase": 1, "total_count": 17}, "IPR016590": {"major tail protein": 3, "total_count": 3}, "IPR006584": {"pectate lyase": 2, "structural protein with Ig domain protein": 20, "virion structural protein": 7, "tail protein with endopeptidase domain protein": 10, "major tail protein": 2, "hypothetical protein": 7, "tail spike protein": 3, "total_count": 51}, "IPR023295": {"Ig domain containing protein": 3, "structural protein with Ig domain protein": 11, "hypothetical protein": 15, "tail sheath": 1, "tail protein": 3, "major tail protein": 5, "regulator of chromosome condensation": 3, "total_count": 41}, "IPR044112": {"HAD-superfamily hydrolase": 2, "HTH-type transcriptional regulator PrtR": 5, "tail protein": 1, "Sulfoquinovosidase": 2, "transcriptional repressor": 1, "hypothetical protein": 1, "Alpha-xylosidase": 1, "total_count": 13}, "IPR000125": {"tail protein": 3, "hypothetical protein": 4, "total_count": 7}, "IPR040527": {"structural protein with Ig domain protein": 12, "major tail protein": 1, "total_count": 13}, "IPR001139": {"structural protein with Ig domain protein": 11, "hypothetical protein": 6, "total_count": 17}, "IPR009470": {"protease": 1, "minor tail protein": 1, "total_count": 2}, "IPR045857": {"Oligo-1,6-glucosidase": 2, "Trehalose-6-phosphate hydrolase": 2, "total_count": 4}, "IPR006421": {"Oligo-1,6-glucosidase": 2, "head maturation protease": 24, "tail fiber protein": 1, "Trehalose-6-phosphate hydrolase": 3, "Glycogen debranching enzyme": 2, "hypothetical protein": 1, "total_count": 33}, "IPR032091": {"Oligo-1,6-glucosidase": 2, "head maturation protease": 24, "Trehalose-6-phosphate hydrolase": 3, "hypothetical protein": 1, "total_count": 30}, "IPR006048": {"Oligo-1,6-glucosidase": 2, "head maturation protease": 24, "Trehalose-6-phosphate hydrolase": 2, "hypothetical protein": 4, "Glycogen debranching enzyme": 2, "total_count": 34}, "IPR036439": {"hypothetical protein": 6, "major tail protein": 4, "zinc carboxypeptidase": 1, "structural protein with Ig domain protein": 1, "total_count": 12}, "IPR027946": {"hypothetical protein": 14, "DNA polymerase exonuclease subunit": 8, "RNA-binding protein": 1, "Hoc-like head decoration": 3, "structural protein with Ig domain protein": 2, "long tail fiber protein distal subunit": 1, "virion structural protein": 1, "total_count": 30}, "IPR000805": {"virion structural protein": 1, "tail fiber assembly": 1, "hypothetical protein": 3, "total_count": 5}, "IPR006103": {"hypothetical protein": 22, "head protein": 22, "Aldose 1-epimerase": 2, "Unsaturated 3S-rhamnoglycuronyl hydrolase": 1, "DNA invertase": 1, "tail fiber protein": 1, "major tail protein": 2, "total_count": 51}, "IPR023933": {"hypothetical protein": 33, "head protein": 23, "Aldose 1-epimerase": 2, "structural protein with Ig domain protein": 1, "Unsaturated 3S-rhamnoglycuronyl hydrolase": 1, "DNA invertase": 1, "tail fiber protein": 1, "major tail protein": 2, "Lactose permease": 1, "tail fiber protein; host specificity": 1, "total_count": 66}, "IPR015177": {"hypothetical protein": 1, "Thymidylate synthase": 2, "major tail protein": 2, "thymidylate synthase": 2, "total_count": 7}, "IPR032312": {"head protein": 22, "DNA invertase": 1, "hypothetical protein": 1, "major tail protein": 1, "total_count": 25}, "IPR024561": {"head maturation protease": 42, "Trehalose-6-phosphate hydrolase": 3, "Glycogen debranching enzyme": 2, "hypothetical protein": 2, "total_count": 49}, "IPR006633": {"tail spike protein": 2, "virion structural protein": 8, "hypothetical protein": 6, "structural protein with Ig domain protein": 1, "tail protein": 3, "total_count": 20}, "IPR022859": {"hypothetical protein": 4, "total_count": 4}, "IPR006425": {"endolysin": 1, "hypothetical protein": 1, "total_count": 2}, "IPR008291": {"DNA helicase": 1, "hypothetical protein": 2, "total_count": 3}, "IPR007781": {"portal protein": 1, "structural protein with Ig domain protein": 1, "hypothetical protein": 1, "total_count": 3}, "IPR006065": {"hypothetical protein": 1, "total_count": 1}, "IPR012939": {"tail protein": 5, "Unsaturated 3S-rhamnoglycuronyl hydrolase": 1, "hypothetical protein": 4, "virion structural protein": 1, "total_count": 11}, "IPR010905": {"hypothetical protein": 3, "Unsaturated 3S-rhamnoglycuronyl hydrolase": 1, "total_count": 4}, "IPR026856": {"tail fiber protein": 6, "hypothetical protein": 12, "photosystem II D1": 3, "Beta-N-acetylhexosaminidase": 1, "Sialidase B": 4, "total_count": 26}, "IPR011040": {"tail fiber protein": 5, "hypothetical protein": 10, "photosystem II D1": 3, "minor tail protein": 2, "Beta-N-acetylhexosaminidase": 1, "Sialidase B": 3, "total_count": 24}, "IPR005200": {"major tail protein": 2, "virion structural protein": 2, "total_count": 4}, "IPR001661": {"Cytoplasmic trehalase": 1, "major tail protein": 1, "hypothetical protein": 1, "glycosyltransferase": 1, "total_count": 4}, "IPR031335": {"Cytoplasmic trehalase": 1, "hypothetical protein": 1, "total_count": 2}, "IPR004888": {"Cytoplasmic trehalase": 2, "total_count": 2}, "IPR032790": {"Cytoplasmic trehalase": 1, "hypothetical protein": 1, "glycosyltransferase": 1, "total_count": 3}, "IPR000922": {"hypothetical protein": 2, "total_count": 2}, "IPR004197": {"tail spike protein": 1, "Cytolethal distending toxin subunit B": 1, "hypothetical protein": 1, "total_count": 3}, "IPR015883": {"tail protein": 1, "Beta-N-acetylhexosaminidase": 1, "hypothetical protein": 1, "total_count": 3}, "IPR000852": {"N-succinylarginine dihydrolase": 1, "total_count": 1}, "IPR046372": {"hypothetical protein": 2, "total_count": 2}, "IPR007724": {"hypothetical protein": 2, "total_count": 2}, "IPR017736": {"6-phospho-beta-glucosidase BglB": 1, "hypothetical protein": 7, "Cryptic outer membrane porin BglH": 1, "integrase": 1, "total_count": 10}, "IPR015165": {"tail sheath": 1, "total_count": 1}, "IPR025092": {"hypothetical protein": 3, "dehydrogenase": 1, "total_count": 4}, "IPR027260": {"2OG-Fe(II) oxygenase": 1, "hypothetical protein": 2, "total_count": 3}, "IPR035394": {"hypothetical protein": 5, "total_count": 5}, "IPR001554": {"hypothetical protein": 3, "total_count": 3}, "IPR011613": {"hypothetical protein": 2, "total_count": 2}, "IPR000165": {"hypothetical protein": 2, "total_count": 2}, "IPR024746": {"hypothetical protein": 2, "glycosyltransferase": 1, "total_count": 3}, "IPR008811": {"tail spike protein": 1, "Alpha-xylosidase": 1, "total_count": 2}, "IPR027291": {"Poly-beta-1,6-N-acetyl-D-glucosamine N-deacetylase": 1, "total_count": 1}, "IPR032979": {"minor tail protein": 1, "total_count": 1}, "IPR005201": {"minor tail protein": 1, "total_count": 1}, "IPR044505": {"Glycogen debranching enzyme": 4, "hypothetical protein": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 6}, "IPR040784": {"Glycogen debranching enzyme": 2, "hypothetical protein": 1, "total_count": 3}, "IPR010702": {"hypothetical protein": 1, "total_count": 1}, "IPR016288": {"hypothetical protein": 1, "total_count": 1}, "IPR036434": {"hypothetical protein": 1, "total_count": 1}, "IPR001137": {"UDP-N-acetyl-D-mannosaminuronic acid transferase": 1, "Ribonucleoside-diphosphate reductase subunit alpha": 1, "total_count": 2}, "IPR012480": {"hypothetical protein": 1, "total_count": 1}, "IPR004185": {"Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 1}}

annotation = {"IPR016283": {"endolysin": 891, "tail fiber protein": 41, "baseplate protein": 17, "hypothetical protein": 24, "structural protein": 6, "chitinase": 2, "tail protein": 4, "baseplate hub subunit and tail lysozyme": 11, "peptidase": 1, "virion structural protein": 10, "baseplate wedge subunit": 1, "total_count": 1008}, "IPR000726": {"endolysin": 725, "tail fiber protein": 41, "baseplate hub subunit and tail lysozyme": 24, "baseplate protein": 17, "hypothetical protein": 23, "structural protein": 6, "metallo-endopeptidase": 3, "chitinase": 2, "tail protein": 3, "peptidase": 1, "virion structural protein": 10, "Low affinity potassium transport system protein kup": 2, "baseplate wedge subunit": 1, "total_count": 858}, "IPR002152": {"internal virion protein with endolysin domain protein": 139, "unknown function": 99, "lytic tail protein": 100, "tRNA nucleotidyltransferase": 5, "tail protein with lysin activity": 14, "endolysin": 32, "transglycosylase": 64, "virion structural protein": 56, "tail fiber protein": 1, "hypothetical protein": 25, "DNA transfer protein": 13, "tail associated lysin": 1, "tail protein": 1, "tail length tape measure protein": 17, "putative transglycosylase IsaA": 3, "polynucleotide kinase": 1, "lytic tail fiber protein": 4, "Membrane-bound lytic murein transglycosylase B": 1, "total_count": 576}, "IPR005604": {"tail needle protein": 24, "tail protein": 261, "hypothetical protein": 245, "minor tail protein": 29, "tail fiber protein": 732, "tail spike protein": 177, "tail collar fiber protein": 35, "virion structural protein": 8, "terminase large subunit": 42, "minor head protein": 9, "haemolysin-type calcium binding protein": 8, "tail fiber assembly": 33, "head decoration": 64, "EPS depolymerase": 11, "glycoside hydrolase": 12, "long tail fiber protein distal subunit": 7, "colanic acid degradation": 13, "tail fiber protein proximal subunit": 1, "hydrolase": 2, "short tail fiber protein": 2, "baseplate wedge subunit": 1, "tail collar protein": 1, "total_count": 1717}, "IPR039473": {"long tail fiber protein proximal connector": 330, "tail protein": 10, "hypothetical protein": 29, "minor head protein": 13, "major tail protein": 3, "neck passage structure": 2, "tail assembly protein": 9, "tail fiber protein and host specificity": 9, "virion structural protein": 2, "glycerophosphoryl diester phosphodiesterase": 2, "tail fiber protein; host specificity": 2, "ParB-like partition protein": 1, "receptor binding tail protein": 2, "FabG-like 3-oxoacyl-(acyl-carrier-protein) reductase": 1, "SbcC-like subunit of palindrome specific endonuclease": 1, "total_count": 416}, "IPR013830": {"virion structural protein": 16, "Hoc-like head decoration": 100, "terminase large subunit": 24, "lipase": 22, "putative protein": 75, "major tail protein": 1, "tail protein": 8, "hypothetical protein": 45, "minor head protein": 9, "neck passage structure protein": 2, "head decoration": 8, "tail fiber protein": 3, "esterase/lipase": 6, "O-acetyltransferase": 1, "tail spike protein": 1, "GDSL lipase": 1, "lysophospholipase L1-like esterase": 1, "Sialidase B": 1, "total_count": 324}, "IPR014635": {"structural protein with Ig domain protein": 5, "DNA methyltransferase": 243, "Periplasmic alpha-amylase": 1, "tail protein": 2, "endolysin": 10, "major tail protein": 14, "FabG-like 3-oxoacyl-(acyl-carrier-protein) reductase": 2, "Oligo-1,6-glucosidase": 2, "head maturation protease": 24, "tail fiber protein": 2, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "hypothetical protein": 7, "DNA adenine methylase": 2, "Catabolic 3-dehydroquinate dehydratase": 1, "hydrolase": 1, "virion structural protein": 2, "Xylose isomerase": 1, "tail fiber protein; host specificity": 1, "Glycogen debranching enzyme": 4, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "total_count": 331}, "IPR013777": {"structural protein with Ig domain protein": 6, "DNA methyltransferase": 241, "Periplasmic alpha-amylase": 1, "tail protein": 6, "baseplate protein": 49, "minor tail protein": 4, "major tail protein": 21, "Oligo-1,6-glucosidase": 2, "hydrolase": 4, "head maturation protease": 29, "tail fiber protein": 5, "tail fiber protein; host specificity": 3, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "hypothetical protein": 10, "Catabolic 3-dehydroquinate dehydratase": 1, "virion structural protein": 2, "endolysin": 1, "Glycogen debranching enzyme": 4, "DNA adenine methylase": 1, "Tn3 family transposase TnAs3": 1, "total_count": 396}, "IPR006046": {"structural protein with Ig domain protein": 5, "DNA methyltransferase": 244, "Periplasmic alpha-amylase": 1, "major tail protein": 25, "tail protein": 5, "baseplate protein": 57, "hypothetical protein": 9, "Oligo-1,6-glucosidase": 2, "hydrolase": 4, "head maturation protease": 24, "tail fiber protein": 2, "Holliday junction resolvase": 1, "tail fiber protein; host specificity": 3, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "terminase large subunit": 1, "endolysin": 1, "Glycogen debranching enzyme": 4, "DNA adenine methylase": 1, "virion structural protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "total_count": 397}, "IPR001724": {"colanic acid degradation": 62, "endolysin": 7, "hypothetical protein": 260, "tail fiber protein": 1058, "EPS depolymerase": 41, "tail spike protein": 593, "tail protein": 216, "long tail fiber protein distal subunit": 80, "major tail protein": 8, "major head protein": 25, "haemolysin-type calcium binding protein": 18, "virion structural protein": 22, "baseplate wedge subunit": 15, "hinge connector of long tail fiber protein distal connector": 7, "pre-neck appendage protein": 23, "head protein": 32, "portal protein": 3, "head scaffolding protein": 2, "baseplate wedge initiator": 5, "terminase large subunit": 9, "tail collar fiber protein": 1, "tail assembly protein": 1, "baseplate hub subunit and tail lysozyme": 18, "minor tail protein": 3, "neck passage structure": 1, "Reverse transcriptase": 1, "tail fiber assembly": 2, "terminase": 1, "short tail fiber protein": 1, "DNA ejection": 2, "tail spike protein with colonic acid degradation activity": 8, "kinase": 1, "total_count": 2526}, "IPR024535": {"endolysin": 7, "hypothetical protein": 363, "minor tail protein": 23, "tail fiber protein": 99, "tail protein": 189, "tail spike protein": 364, "virion structural protein": 26, "portal protein": 6, "major head protein": 5, "EPS depolymerase": 47, "receptor binding tail protein": 5, "pre-neck appendage protein": 79, "head scaffolding protein": 2, "terminase large subunit": 9, "colanic acid degradation": 34, "tail collar protein": 4, "neck passage structure": 2, "Hoc-like head decoration": 1, "tRNA-His guanylyltransferase": 1, "terminase": 1, "head decoration": 2, "structural protein with Ig domain protein": 3, "major tail protein": 2, "adaptor Ad4": 2, "total_count": 1276}, "IPR000933": {"tail protein": 11, "structural protein with Ig domain protein": 9, "major tail protein": 1, "hypothetical protein": 1, "total_count": 22}, "IPR016286": {"tail protein": 11, "structural protein with Ig domain protein": 9, "major tail protein": 1, "hypothetical protein": 1, "total_count": 22}, "IPR003790": {"tail protein": 14, "Pyridoxine 5'-phosphate synthase": 1, "Oligo-1,6-glucosidase": 2, "minor tail protein": 7, "major tail protein": 1, "pseudomurein endosiopeptidase": 1, "structural protein with Ig domain protein": 1, "Trehalose-6-phosphate hydrolase": 1, "hypothetical protein": 8, "lipase": 3, "Sulfoquinovosidase": 2, "hydrolase": 1, "Germination-specific N-acetylmuramoyl-L-alanine amidase": 1, "Beta-N-acetylhexosaminidase": 1, "SbcD-like subunit of palindrome specific endonuclease": 1, "Poly-beta-1,6-N-acetyl-D-glucosamine N-deacetylase": 1, "Glycogen debranching enzyme": 3, "tail fiber protein": 1, "Alpha-xylosidase": 1, "Hoc-like head decoration": 1, "total_count": 52}, "IPR000974": {"anti-repressor Ant": 59, "virion structural protein": 10, "hypothetical protein": 16, "tail length tape measure protein": 34, "putative transglycosylase IsaA": 22, "anti-repressor": 32, "tail fiber protein": 8, "transglycosylase": 16, "polynucleotide kinase": 1, "baseplate hub subunit and tail lysozyme": 1, "putative protein YqbO": 1, "total_count": 200}, "IPR005193": {"tail sheath": 135, "tail protein": 3, "queuine tRNA-ribosyltransferase": 2, "total_count": 140}, "IPR044914": {"tail fiber protein": 76, "tail spike protein": 75, "hypothetical protein": 64, "tail protein": 24, "minor tail protein": 1, "virion structural protein": 3, "baseplate wedge initiator": 1, "colanic acid degradation": 1, "total_count": 245}, "IPR024430": {"tail fiber protein": 44, "baseplate wedge subunit": 9, "tail spike protein": 62, "hypothetical protein": 20, "EPS depolymerase": 1, "tail protein": 17, "colanic acid degradation": 1, "total_count": 154}, "IPR039279": {"hypothetical protein": 362, "tail fiber protein": 104, "EPS depolymerase": 53, "tail protein": 168, "tail spike protein": 352, "virion structural protein": 25, "minor tail protein": 27, "receptor binding tail protein": 7, "pre-neck appendage protein": 80, "colanic acid degradation": 35, "tail collar protein": 4, "neck passage structure": 2, "pectin lyase activity": 1, "tRNA-His guanylyltransferase": 1, "head decoration": 1, "adaptor Ad4": 2, "total_count": 1224}, "IPR000514": {"hypothetical protein": 12, "hydrolase": 2, "minor tail protein": 1, "transcriptional regulator": 2, "6-phospho-beta-glucosidase BglB": 1, "major tail protein": 2, "total_count": 20}, "IPR001000": {"hypothetical protein": 7, "baseplate wedge tail fiber protein connector": 1, "6-phospho-beta-glucosidase BglB": 1, "Exoglucanase-2": 1, "tail spike protein": 1, "total_count": 11}, "IPR024745": {"hypothetical protein": 5, "5,10-methylenetetrahydrofolate reductase": 1, "total_count": 6}, "IPR003476": {"hypothetical protein": 14, "baseplate protein": 61, "tail protein": 8, "Oligo-1,6-glucosidase": 2, "hydrolase": 5, "L-ribulose-5-phosphate 3-epimerase UlaE": 2, "radical SAM domain-containing protein": 2, "Trehalose-6-phosphate hydrolase": 2, "DNA invertase": 1, "6-phospho-beta-glucosidase BglB": 1, "lipase": 3, "Peptidoglycan deacetylase": 1, "major tail protein": 1, "tail fiber protein": 1, "Sugar phosphatase YidA": 1, "total_count": 105}, "IPR011683": {"hypothetical protein": 8, "major tail protein": 9, "Oligo-1,6-glucosidase": 2, "hydrolase": 4, "RNA ligase and tail fiber protein attachment catalyst": 1, "head maturation protease": 1, "structural protein with Ig domain protein": 5, "tail protein": 4, "6-phospho-beta-glucosidase BglB": 1, "Trehalose-6-phosphate hydrolase": 2, "transglycosylase": 1, "Biotin synthase": 1, "total_count": 39}, "IPR005199": {"hypothetical protein": 6, "Guanine deaminase": 1, "total_count": 7}, "IPR001547": {"hypothetical protein": 12, "baseplate protein": 61, "RNA ligase": 28, "Oligo-1,6-glucosidase": 2, "hydrolase": 4, "Trehalose-6-phosphate hydrolase": 3, "tail protein": 4, "6-phospho-beta-glucosidase BglB": 1, "virion structural protein": 1, "endolysin": 1, "total_count": 117}, "IPR044846": {"hypothetical protein": 1, "Exoglucanase-2": 1, "total_count": 2}, "IPR039743": {"hypothetical protein": 7, "minor tail protein": 1, "major tail protein": 2, "total_count": 10}, "IPR039514": {"hypothetical protein": 2, "minor tail protein": 1, "major tail protein": 1, "total_count": 4}, "IPR021865": {"minor tail protein": 75, "tail protein": 11, "pre-neck appendage protein": 30, "tail fiber protein": 24, "hypothetical protein": 37, "virion structural protein": 23, "tail spike protein": 17, "tail assembly protein": 1, "zinc carboxypeptidase": 3, "neck passage structure": 1, "colanic acid degradation": 1, "total_count": 223}, "IPR008585": {"tail protein": 18, "minor tail protein": 157, "hypothetical protein": 36, "poly-gamma-glutamate hydrolase": 19, "glycerophosphoryl diester phosphodiesterase": 14, "tail tip protein Tal": 1, "tail fiber protein": 1, "Sulfite reductase, dissimilatory-type subunit alpha": 2, "total_count": 248}, "IPR002925": {"endolysin": 9, "Vitamin B12-dependent ribonucleoside-diphosphate reductase": 22, "esterase/lipase": 56, "hypothetical protein": 46, "Non-heme chloroperoxidase": 146, "Modification methylase RsrI": 4, "esterase": 27, "Haloacetate dehalogenase H-1": 2, "Arylesterase": 79, "Pimeloyl-[acyl-carrier protein] methyl ester esterase": 2, "Lipase 2": 2, "Modification methylase DpnIIB": 5, "peptidase": 5, "DNA methyltransferase": 3, "ribonucleotide reductase large subunit": 1, "RNA polymerase": 2, "Putative acetyl-hydrolase LipR": 1, "DNA polymerase exonuclease subunit": 8, "putative protein": 15, "ribonucleotide reductase": 5, "virion structural protein": 1, "Poly(3-hydroxyalkanoate) polymerase subunit PhaC": 4, "Aclacinomycin methylesterase RdmC": 2, "Modification methylase MboII": 1, "long tail fiber protein distal subunit": 1, "Lysophospholipase L2": 2, "DNA helicase": 1, "nucleotide-sugar epimerase": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 454}, "IPR024428": {"tail spike protein": 27, "tail fiber protein": 35, "hypothetical protein": 26, "major head protein": 21, "tail protein": 12, "EPS depolymerase": 2, "total_count": 123}, "IPR029456": {"hypothetical protein": 15, "tail fiber assembly": 3, "tail protein": 1, "major tail protein": 8, "Hoc-like head decoration": 1, "single strand DNA binding protein": 1, "total_count": 29}, "IPR024429": {"tail spike protein": 26, "tail protein": 8, "hypothetical protein": 15, "pre-neck appendage protein": 11, "tail fiber protein": 5, "virion structural protein": 1, "minor tail protein": 3, "colanic acid degradation": 2, "total_count": 71}, "IPR000490": {"UvsX-like recombinase": 92, "Protein RecA": 12, "total_count": 104}, "IPR023720": {"DNA repair protein": 70, "UvsX-like recombinase": 13, "Protein RecA": 16, "recombinase": 1, "Cytoplasmic trehalase": 1, "hypothetical protein": 2, "total_count": 103}, "IPR015331": {"tail spike protein": 220, "colanic acid degradation": 2, "tail protein": 15, "hypothetical protein": 6, "tail fiber protein": 24, "haemolysin-type calcium binding protein": 3, "total_count": 270}, "IPR016828": {"tail protein": 10, "major tail protein": 62, "structural protein with Ig domain protein": 8, "tail fiber protein; host specificity": 1, "Inner membrane symporter YicJ": 1, "Extracellular exo-alpha-(1->5)-L-arabinofuranosidase": 1, "lipase": 1, "total_count": 84}, "IPR026283": {"tail protein": 12, "hydrolase": 4, "2-amino-3-ketobutyrate coenzyme A ligase": 3, "hypothetical protein": 4, "major tail protein": 1, "total_count": 24}, "IPR037110": {"tail protein": 4, "hypothetical protein": 1, "total_count": 5}, "IPR031330": {"tail protein": 8, "hydrolase": 4, "2-amino-3-ketobutyrate coenzyme A ligase": 3, "hypothetical protein": 3, "total_count": 18}, "IPR038964": {"tail protein": 13, "hypothetical protein": 1, "virion structural protein": 10, "total_count": 24}, "IPR006710": {"tail protein": 9, "Levanbiose-producing levanase": 1, "hypothetical protein": 2, "structural protein with Ig domain protein": 8, "major tail protein": 1, "Inner membrane symporter YicJ": 1, "Extracellular exo-alpha-(1->5)-L-arabinofuranosidase": 1, "levanase": 4, "lipase": 1, "total_count": 28}, "IPR016840": {"tail protein": 14, "Hoc-like head decoration": 1, "Levanbiose-producing levanase": 1, "hypothetical protein": 3, "structural protein with Ig domain protein": 5, "Inner membrane symporter YicJ": 1, "Extracellular exo-alpha-(1->5)-L-arabinofuranosidase": 1, "virion structural protein": 1, "levanase": 2, "lipase": 1, "major tail protein": 1, "total_count": 31}, "IPR043534": {"tail protein": 8, "hypothetical protein": 27, "head protein": 22, "Aldose 1-epimerase": 2, "Unsaturated 3S-rhamnoglycuronyl hydrolase": 1, "DNA invertase": 1, "virion structural protein": 2, "tail fiber protein": 1, "major tail protein": 2, "Hoc-like head decoration": 1, "total_count": 67}, "IPR006101": {"tail protein": 8, "fibritin neck whisker": 4, "hypothetical protein": 29, "head protein": 22, "hydrolase": 4, "Aldose 1-epimerase": 2, "structural protein with Ig domain protein": 2, "Adenosylmethionine-8-amino-7-oxononanoate aminotransferase": 1, "tail fiber protein; host specificity": 3, "Unsaturated 3S-rhamnoglycuronyl hydrolase": 1, "DNA invertase": 1, "aminotransferase": 1, "tail fiber protein": 1, "major tail protein": 3, "Lactose permease": 1, "Hoc-like head decoration": 1, "total_count": 84}, "IPR003469": {"tail protein": 4, "Levanbiose-producing levanase": 1, "hypothetical protein": 1, "levanase": 2, "total_count": 8}, "IPR016007": {"tail protein": 4, "minor tail protein": 54, "virion structural protein": 15, "hypothetical protein": 9, "neck passage structure protein": 2, "Cytoplasmic trehalase": 1, "major tail protein": 2, "major head protein": 1, "total_count": 88}, "IPR001362": {"tail protein": 6, "Levanbiose-producing levanase": 1, "levanase": 2, "hypothetical protein": 1, "tail fiber protein": 1, "total_count": 11}, "IPR013148": {"tail protein": 5, "Levanbiose-producing levanase": 1, "structural protein with Ig domain protein": 2, "levanase": 2, "tail fiber protein": 1, "total_count": 11}, "IPR009860": {"hyaluronidase": 47, "hypothetical protein": 56, "tail fiber protein": 17, "hinge connector of long tail fiber protein distal connector": 1, "tail collar fiber protein": 1, "tail fiber protein proximal subunit": 3, "total_count": 125}, "IPR016289": {"SbcD-like subunit of palindrome specific endonuclease": 45, "minor tail protein": 65, "exonuclease": 8, "hydrolase": 4, "major tail protein": 13, "hypothetical protein": 12, "virion structural protein": 2, "endolysin": 1, "tail protein": 1, "total_count": 151}, "IPR034641": {"major tail protein": 75, "Ig domain containing protein": 1, "minor tail protein": 6, "tail protein": 16, "hypothetical protein": 7, "head maturation protease": 7, "virion structural protein": 2, "head closure Hc2": 3, "baseplate hub protein Tal": 4, "minor head protein": 2, "total_count": 123}, "IPR041624": {"major tail protein": 29, "hypothetical protein": 39, "virion structural protein": 11, "Exoglucanase-2": 5, "minor tail protein": 87, "Ig domain containing protein": 3, "S-(hydroxymethyl)mycothiol dehydrogenase": 1, "baseplate hub protein Tal": 4, "tail fiber protein": 1, "esterase/lipase": 1, "tail protein": 3, "endolysin": 2, "total_count": 186}, "IPR017069": {"Periplasmic alpha-amylase": 1, "tail protein": 2, "major tail protein": 9, "Oligo-1,6-glucosidase": 2, "structural protein with Ig domain protein": 13, "head maturation protease": 24, "tail fiber protein": 2, "nicotinamide phosphoribosyl transferase": 6, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "hypothetical protein": 10, "endolysin": 1, "Glycogen debranching enzyme": 4, "DNA adenine methylase": 1, "virion structural protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "Lipopolysaccharide core heptosyltransferase RfaQ": 1, "UvsX-like recombinase": 1, "total_count": 85}, "IPR006047": {"Periplasmic alpha-amylase": 1, "tail protein": 2, "major tail protein": 10, "Oligo-1,6-glucosidase": 2, "structural protein with Ig domain protein": 4, "head maturation protease": 24, "tail fiber protein": 2, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "hydrolase": 1, "hypothetical protein": 8, "terminase large subunit": 1, "endolysin": 1, "tail fiber protein; host specificity": 2, "Glycogen debranching enzyme": 4, "virion structural protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "total_count": 70}, "IPR013776": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "major tail protein": 19, "Oligo-1,6-glucosidase": 2, "Thymidylate synthase": 4, "head maturation protease": 25, "tail fiber protein": 2, "thymidylate synthase": 2, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "Thymidylate synthase 2": 1, "hypothetical protein": 6, "structural protein with Ig domain protein": 2, "tail fiber protein; host specificity": 1, "Glycogen debranching enzyme": 4, "virion structural protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "total_count": 80}, "IPR005323": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "major tail protein": 9, "hypothetical protein": 14, "Oligo-1,6-glucosidase": 2, "head maturation protease": 48, "tail fiber protein": 7, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 5, "virion structural protein": 3, "Glycogen debranching enzyme": 4, "structural protein with Ig domain protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 102}, "IPR011837": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "major tail protein": 9, "Oligo-1,6-glucosidase": 2, "head maturation protease": 40, "tail fiber protein": 5, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 4, "hypothetical protein": 9, "structural protein with Ig domain protein": 2, "Glycogen debranching enzyme": 4, "virion structural protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 85}, "IPR011840": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "major tail protein": 11, "hypothetical protein": 14, "Oligo-1,6-glucosidase": 2, "head maturation protease": 52, "tail fiber protein": 8, "tail fiber protein; host specificity": 1, "QueE-like radical SAM domain protein": 1, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "virion structural protein": 3, "Glycogen debranching enzyme": 4, "DnaB-like replicative helicase": 1, "structural protein with Ig domain protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 110}, "IPR011838": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "hypothetical protein": 13, "major tail protein": 7, "Oligo-1,6-glucosidase": 2, "head maturation protease": 48, "tail fiber protein": 6, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "virion structural protein": 2, "Glycogen debranching enzyme": 4, "structural protein with Ig domain protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 95}, "IPR013775": {"Periplasmic alpha-amylase": 1, "tail protein": 1, "major tail protein": 10, "Oligo-1,6-glucosidase": 2, "head maturation protease": 24, "tail fiber protein": 2, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "hypothetical protein": 3, "Glycogen debranching enzyme": 4, "virion structural protein": 1, "structural protein with Ig domain protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "total_count": 56}, "IPR022844": {"Periplasmic alpha-amylase": 1, "tail protein": 2, "major tail protein": 6, "Oligo-1,6-glucosidase": 2, "unknown function": 19, "head maturation protease": 40, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "hypothetical protein": 6, "tail fiber protein": 4, "Glycogen debranching enzyme": 4, "virion structural protein": 1, "structural protein with Ig domain protein": 1, "minor tail protein": 1, "Tn3 family transposase TnAs3": 1, "total_count": 93}, "IPR011839": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "major tail protein": 6, "hypothetical protein": 10, "Oligo-1,6-glucosidase": 2, "endolysin": 2, "head maturation protease": 48, "tail fiber protein": 7, "Tn3 family transposase TnAs1": 2, "Trehalose-6-phosphate hydrolase": 3, "virion structural protein": 4, "Glycogen debranching enzyme": 4, "structural protein with Ig domain protein": 1, "Tn3 family transposase TnAs3": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 95}, "IPR032792": {"Periplasmic alpha-amylase": 1, "Oligo-1,6-glucosidase": 2, "head maturation protease": 24, "tail fiber protein": 1, "DNA polymerase exonuclease subunit": 14, "Trehalose-6-phosphate hydrolase": 3, "major tail protein": 1, "hypothetical protein": 2, "Glycogen debranching enzyme": 2, "virion structural protein": 1, "structural protein with Ig domain protein": 1, "total_count": 52}, "IPR002252": {"Periplasmic alpha-amylase": 1, "tail protein": 3, "Oligo-1,6-glucosidase": 2, "Tn3 family transposase TnAs1": 2, "hypothetical protein": 3, "Trehalose-6-phosphate hydrolase": 3, "Sulfoquinovosidase": 2, "Germination-specific N-acetylmuramoyl-L-alanine amidase": 1, "tail spike protein": 2, "Biotin synthase": 1, "Lactose permease": 1, "Alpha-xylosidase": 1, "Tn3 family transposase TnAs3": 1, "total_count": 23}, "IPR019563": {"hypothetical protein": 7, "deoxynucleoside monophosphate kinase": 2, "4-hydroxy-3-methylbut-2-en-1-yl diphosphate synthase (flavodoxin)": 3, "radical SAM domain-containing protein": 2, "tail fiber protein": 2, "Trehalose-6-phosphate hydrolase": 2, "Glycerophosphodiester phosphodiesterase": 1, "Catabolic 3-dehydroquinate dehydratase": 1, "Purine nucleoside phosphorylase DeoD-type": 2, "3-dehydroquinate dehydratase": 1, "L-ribulose-5-phosphate 3-epimerase UlaE": 1, "total_count": 24}, "IPR037398": {"major tail protein": 5, "hypothetical protein": 6, "Hoc-like head decoration": 3, "tail protein": 4, "fibritin neck whisker": 10, "total_count": 28}, "IPR026071": {"hypothetical protein": 6, "4-sulfomuconolactone hydrolase": 2, "2-amino-3-ketobutyrate coenzyme A ligase": 3, "radical SAM domain-containing protein": 3, "homing endonuclease": 1, "total_count": 15}, "IPR012970": {"major tail protein": 42, "tail fiber protein; host specificity": 2, "hypothetical protein": 3, "structural protein with Ig domain protein": 6, "total_count": 53}, "IPR011395": {"tail protein": 7, "Beta-N-acetylhexosaminidase": 1, "hypothetical protein": 1, "total_count": 9}, "IPR011100": {"tail protein": 7, "total_count": 7}, "IPR006215": {"tail protein": 9, "tail spike protein": 3, "hypothetical protein": 1, "total_count": 13}, "IPR000743": {"tail protein": 3, "hypothetical protein": 6, "tail spike protein": 2, "pre-neck appendage protein": 1, "total_count": 12}, "IPR002241": {"tail protein": 5, "tail spike protein": 3, "total_count": 8}, "IPR039513": {"hypothetical protein": 21, "transcriptional repressor": 10, "major tail protein": 1, "virion structural protein": 4, "Hoc-like head decoration": 1, "Holliday junction ATP-dependent DNA helicase RuvB": 1, "DNA helicase": 1, "tail fiber protein": 2, "structural protein with Ig domain protein": 2, "Ribosomal RNA small subunit methyltransferase E": 2, "Ferredoxin--NADP reductase": 1, "hydrolase": 1, "neck protein": 2, "Stage V sporulation protein K": 1, "tail spike protein": 1, "protease": 1, "distal tail protein": 1, "total_count": 53}, "IPR000400": {"baseplate hub subunit and tail lysozyme": 39, "hypothetical protein": 45, "endolysin": 5, "dehydrogenase": 1, "baseplate protein": 1, "total_count": 91}, "IPR011583": {"minor tail protein": 66, "protease": 1, "tail protein": 5, "hypothetical protein": 17, "endolysin": 1, "major tail protein": 3, "Dda-like helicase": 1, "total_count": 94}, "IPR001223": {"minor tail protein": 66, "protease": 1, "tail protein": 5, "hypothetical protein": 15, "endolysin": 1, "major tail protein": 3, "Dda-like helicase": 1, "total_count": 92}, "IPR000677": {"minor tail protein": 66, "major tail protein": 10, "protease": 1, "hydrolase": 5, "tail protein": 6, "hypothetical protein": 17, "Urease accessory protein UreG": 1, "endolysin": 1, "Cytolethal distending toxin subunit B": 1, "Dda-like helicase": 1, "tail spike protein": 1, "SbcD-like subunit of palindrome specific endonuclease": 1, "head maturation protease": 1, "total_count": 112}, "IPR038970": {"structural protein with Ig domain protein": 8, "major tail protein": 23, "hypothetical protein": 2, "tail fiber protein; host specificity": 1, "3-(3-hydroxy-phenyl)propionate/3-hydroxycinnamic acid hydroxylase": 1, "total_count": 35}, "IPR045032": {"tail protein": 8, "tail spike protein": 55, "pectate lyase": 3, "pre-neck appendage protein": 26, "major tail protein": 7, "hypothetical protein": 22, "hydrolase": 4, "virion structural protein": 1, "minor tail protein": 1, "total_count": 127}, "IPR018082": {"tail protein": 6, "tail spike protein": 30, "Hoc-like head decoration": 4, "pectate lyase": 3, "pre-neck appendage protein": 26, "tail fiber protein": 2, "major tail protein": 7, "hypothetical protein": 16, "hydrolase": 4, "tail assembly protein": 3, "virion structural protein": 4, "total_count": 105}, "IPR002022": {"tail protein": 6, "tail spike protein": 35, "pectate lyase": 3, "pre-neck appendage protein": 26, "major tail protein": 7, "hypothetical protein": 21, "hydrolase": 4, "virion structural protein": 1, "tail fiber protein": 1, "total_count": 104}, "IPR025975": {"peptidase": 8, "hypothetical protein": 63, "virion structural protein": 1, "endolysin": 1, "total_count": 73}, "IPR011496": {"peptidase": 61, "Sak4-like ssDNA annealing protein": 4, "Hydroxymethylglutaryl-CoA lyase YngG": 2, "hypothetical protein": 1, "total_count": 68}, "IPR008397": {"ribonucleoside diphosphate reductase small subunit": 2, "Ribonucleoside-diphosphate reductase 2 subunit beta": 1, "hypothetical protein": 6, "tail fiber protein": 2, "total_count": 11}, "IPR008929": {"ribonucleoside diphosphate reductase small subunit": 2, "hypothetical protein": 8, "Ribonucleoside-diphosphate reductase 2 subunit beta": 1, "total_count": 11}, "IPR016455": {"glucosyltransferase": 17, "hypothetical protein": 15, "total_count": 32}, "IPR016287": {"glucosyltransferase": 14, "hypothetical protein": 37, "Ig domain containing protein": 1, "major tail protein": 2, "tail protein": 1, "structural protein with Ig domain protein": 2, "virion structural protein": 1, "total_count": 58}, "IPR000757": {"glucosyltransferase": 14, "hypothetical protein": 14, "Ig domain containing protein": 1, "structural protein with Ig domain protein": 1, "total_count": 30}, "IPR016282": {"baseplate protein": 61, "major tail protein": 10, "hydrolase": 4, "Ribonucleoside-diphosphate reductase subunit alpha": 1, "hypothetical protein": 3, "tail protein": 6, "tail fiber protein": 2, "endolysin": 1, "total_count": 88}, "IPR000556": {"major tail protein": 17, "virion structural protein": 5, "Exoglucanase-2": 3, "tail protein": 31, "Ig domain containing protein": 1, "minor tail protein": 4, "hypothetical protein": 3, "tail assembly protein": 3, "endolysin": 1, "total_count": 68}, "IPR025706": {"structural protein with Ig domain protein": 1, "tail fiber protein; host specificity": 7, "Oligo-1,6-glucosidase": 2, "major tail protein": 1, "tail protein": 1, "Trehalose-6-phosphate hydrolase": 1, "total_count": 13}, "IPR024200": {"major tail protein": 13, "hypothetical protein": 1, "tail fiber protein; host specificity": 2, "Thymidylate synthase": 2, "endolysin": 1, "thymidylate synthase": 1, "total_count": 20}, "IPR003159": {"major tail protein": 20, "hypothetical protein": 3, "tail fiber protein; host specificity": 1, "structural protein with Ig domain protein": 2, "total_count": 26}, "IPR039174": {"major tail protein": 8, "hypothetical protein": 1, "tail fiber protein; host specificity": 2, "endolysin": 1, "total_count": 12}, "IPR014895": {"hypothetical protein": 10, "total_count": 10}, "IPR018155": {"hypothetical protein": 1, "minor tail protein with lysin activity": 1, "2OG-Fe(II) oxygenase": 1, "Methionine synthase": 1, "total_count": 4}, "IPR001329": {"hypothetical protein": 3, "2OG-Fe(II) oxygenase": 1, "Homoserine O-succinyltransferase": 1, "total_count": 5}, "IPR001088": {"UDP-glucose dehydrogenase": 11, "Cell division protein FtsA": 2, "hypothetical protein": 15, "N-acetyl-gamma-glutamyl-phosphate reductase": 1, "FabG-like 3-oxoacyl-(acyl-carrier-protein) reductase": 6, "UDP-glucose 6-dehydrogenase TuaD": 2, "3-hydroxyadipyl-CoA dehydrogenase": 2, "4-hydroxy-tetrahydrodipicolinate reductase": 2, "dehydrogenase": 5, "Lysine 6-dehydrogenase": 1, "Malate dehydrogenase": 1, "Pyrroline-5-carboxylate reductase": 2, "Cell division protein FtsZ": 2, "Sulfofructosephosphate aldolase": 1, "6-phospho-alpha-glucosidase": 1, "Carbamoyl-phosphate synthase small chain": 1, "Alpha-galactosidase": 1, "DNA gyrase subunit B": 3, "ribonucleotide reductase large subunit": 1, "Glycerol-3-phosphate dehydrogenase [NAD(P)+]": 3, "NADPH oxidoreductase": 1, "tryptophan halogenase": 1, "Flavin-dependent tryptophan halogenase RebH": 1, "Chromosome-partitioning ATPase Soj": 1, "Heme response regulator HssR": 1, "Fatty acid oxidation complex subunit alpha": 2, "Phosphoribosylformylglycinamidine synthase subunit PurQ": 1, "UDP-N-acetylglucosamine 4,6-dehydratase (inverting)": 1, "DNA topoisomerase II large subunit": 1, "UDP-glucose 6-dehydrogenase": 1, "metal-dependent hydrolase": 1, "total_count": 75}, "IPR002037": {"major tail protein": 5, "hypothetical protein": 2, "structural protein with Ig domain protein": 12, "total_count": 19}, "IPR023309": {"Exoglucanase-2": 3, "major tail protein": 1, "total_count": 4}, "IPR012878": {"minor tail protein": 1, "glycosyltransferase": 9, "hypothetical protein": 4, "total_count": 14}, "IPR016714": {"hypothetical protein": 8, "tail protein": 3, "virion structural protein": 1, "tail fiber assembly": 1, "total_count": 13}, "IPR041351": {"hypothetical protein": 9, "virion structural protein": 1, "total_count": 10}, "IPR041542": {"hypothetical protein": 4, "fibritin neck whisker": 10, "tail fiber protein": 1, "total_count": 15}, "IPR001286": {"hypothetical protein": 9, "major tail protein": 4, "tail fiber protein": 4, "tail spike protein": 1, "tail protein": 2, "total_count": 20}, "IPR006775": {"tail fiber protein": 2, "neck passage structure protein": 2, "endolysin": 1, "hypothetical protein": 3, "glycosyltransferase": 1, "total_count": 9}, "IPR014551": {"tail fiber protein": 2, "neck passage structure protein": 2, "endolysin": 1, "hypothetical protein": 2, "glycosyltransferase": 1, "total_count": 8}, "IPR005198": {"tail fiber protein": 2, "hypothetical protein": 8, "Sulfoquinovose isomerase": 2, "Unsaturated 3S-rhamnoglycuronyl hydrolase": 1, "tail protein": 2, "minor tail protein": 1, "total_count": 16}, "IPR035396": {"tail fiber protein": 2, "neck passage structure protein": 2, "hypothetical protein": 10, "minor tail protein": 1, "Cytoplasmic trehalase": 1, "major tail protein": 1, "glycosyltransferase": 1, "major head protein": 1, "total_count": 19}, "IPR036881": {"tail protein": 2, "hypothetical protein": 17, "virion structural protein": 1, "total_count": 20}, "IPR015289": {"tail protein": 8, "hypothetical protein": 7, "virion structural protein": 13, "minor tail protein": 1, "tail fiber assembly": 1, "baseplate wedge tail fiber protein connector": 1, "total_count": 31}, "IPR001371": {"hypothetical protein": 5, "total_count": 5}, "IPR021016": {"minor tail protein": 1, "total_count": 1}, "IPR008902": {"neck passage structure protein": 2, "hypothetical protein": 1, "major head protein": 1, "total_count": 4}, "IPR001439": {"hypothetical protein": 14, "2OG-Fe(II) oxygenase": 2, "acetyltransferase": 1, "total_count": 17}, "IPR013189": {"Levanbiose-producing levanase": 1, "major tail protein": 1, "levanase": 2, "hypothetical protein": 1, "total_count": 5}, "IPR016590": {"major tail protein": 3, "total_count": 3}, "IPR039448": {"tail spike protein": 17, "hypothetical protein": 3, "total_count": 20}, "IPR006584": {"pectate lyase": 2, "structural protein with Ig domain protein": 20, "virion structural protein": 7, "tail protein with endopeptidase domain protein": 10, "major tail protein": 2, "hypothetical protein": 7, "tail spike protein": 3, "total_count": 51}, "IPR023295": {"Ig domain containing protein": 3, "structural protein with Ig domain protein": 11, "hypothetical protein": 15, "tail sheath": 1, "tail protein": 3, "major tail protein": 5, "regulator of chromosome condensation": 3, "total_count": 41}, "IPR044112": {"HAD-superfamily hydrolase": 2, "HTH-type transcriptional regulator PrtR": 5, "tail protein": 1, "Sulfoquinovosidase": 2, "transcriptional repressor": 1, "hypothetical protein": 1, "Alpha-xylosidase": 1, "total_count": 13}, "IPR000125": {"tail protein": 3, "hypothetical protein": 4, "total_count": 7}, "IPR040527": {"structural protein with Ig domain protein": 12, "major tail protein": 1, "total_count": 13}, "IPR001139": {"structural protein with Ig domain protein": 11, "hypothetical protein": 6, "total_count": 17}, "IPR009470": {"protease": 1, "minor tail protein": 1, "total_count": 2}, "IPR006421": {"Oligo-1,6-glucosidase": 2, "head maturation protease": 24, "tail fiber protein": 1, "Trehalose-6-phosphate hydrolase": 3, "Glycogen debranching enzyme": 2, "hypothetical protein": 1, "total_count": 33}, "IPR032091": {"Oligo-1,6-glucosidase": 2, "head maturation protease": 24, "Trehalose-6-phosphate hydrolase": 3, "hypothetical protein": 1, "total_count": 30}, "IPR006048": {"Oligo-1,6-glucosidase": 2, "head maturation protease": 24, "Trehalose-6-phosphate hydrolase": 2, "hypothetical protein": 4, "Glycogen debranching enzyme": 2, "total_count": 34}, "IPR036439": {"hypothetical protein": 6, "major tail protein": 4, "zinc carboxypeptidase": 1, "structural protein with Ig domain protein": 1, "total_count": 12}, "IPR027946": {"hypothetical protein": 14, "DNA polymerase exonuclease subunit": 8, "RNA-binding protein": 1, "Hoc-like head decoration": 3, "structural protein with Ig domain protein": 2, "long tail fiber protein distal subunit": 1, "virion structural protein": 1, "total_count": 30}, "IPR000805": {"virion structural protein": 1, "tail fiber assembly": 1, "hypothetical protein": 3, "total_count": 5}, "IPR006103": {"hypothetical protein": 22, "head protein": 22, "Aldose 1-epimerase": 2, "Unsaturated 3S-rhamnoglycuronyl hydrolase": 1, "DNA invertase": 1, "tail fiber protein": 1, "major tail protein": 2, "total_count": 51}, "IPR023933": {"hypothetical protein": 33, "head protein": 23, "Aldose 1-epimerase": 2, "structural protein with Ig domain protein": 1, "Unsaturated 3S-rhamnoglycuronyl hydrolase": 1, "DNA invertase": 1, "tail fiber protein": 1, "major tail protein": 2, "Lactose permease": 1, "tail fiber protein; host specificity": 1, "total_count": 66}, "IPR015177": {"hypothetical protein": 1, "Thymidylate synthase": 2, "major tail protein": 2, "thymidylate synthase": 2, "total_count": 7}, "IPR032312": {"head protein": 22, "DNA invertase": 1, "hypothetical protein": 1, "major tail protein": 1, "total_count": 25}, "IPR024561": {"head maturation protease": 42, "Trehalose-6-phosphate hydrolase": 3, "Glycogen debranching enzyme": 2, "hypothetical protein": 2, "total_count": 49}, "IPR006633": {"tail spike protein": 2, "virion structural protein": 8, "hypothetical protein": 6, "structural protein with Ig domain protein": 1, "tail protein": 3, "total_count": 20}, "IPR022859": {"hypothetical protein": 4, "total_count": 4}, "IPR035669": {"ATPase": 4, "hypothetical protein": 3, "total_count": 7}, "IPR006425": {"endolysin": 1, "hypothetical protein": 1, "total_count": 2}, "IPR008291": {"DNA helicase": 1, "hypothetical protein": 2, "total_count": 3}, "IPR007781": {"portal protein": 1, "structural protein with Ig domain protein": 1, "hypothetical protein": 1, "total_count": 3}, "IPR006065": {"hypothetical protein": 1, "total_count": 1}, "IPR012939": {"tail protein": 5, "Unsaturated 3S-rhamnoglycuronyl hydrolase": 1, "hypothetical protein": 4, "virion structural protein": 1, "total_count": 11}, "IPR000322": {"Isoprimeverose transporter": 1, "tail protein": 1, "Sulfoquinovosidase": 2, "hypothetical protein": 1, "Alpha-xylosidase": 1, "total_count": 6}, "IPR010905": {"hypothetical protein": 3, "Unsaturated 3S-rhamnoglycuronyl hydrolase": 1, "total_count": 4}, "IPR026856": {"tail fiber protein": 6, "hypothetical protein": 12, "photosystem II D1": 3, "Beta-N-acetylhexosaminidase": 1, "Sialidase B": 4, "total_count": 26}, "IPR011040": {"tail fiber protein": 5, "hypothetical protein": 10, "photosystem II D1": 3, "minor tail protein": 2, "Beta-N-acetylhexosaminidase": 1, "Sialidase B": 3, "total_count": 24}, "IPR005200": {"major tail protein": 2, "virion structural protein": 2, "total_count": 4}, "IPR001661": {"Cytoplasmic trehalase": 1, "major tail protein": 1, "hypothetical protein": 1, "glycosyltransferase": 1, "total_count": 4}, "IPR031335": {"Cytoplasmic trehalase": 1, "hypothetical protein": 1, "total_count": 2}, "IPR004888": {"Cytoplasmic trehalase": 2, "total_count": 2}, "IPR032790": {"Cytoplasmic trehalase": 1, "hypothetical protein": 1, "glycosyltransferase": 1, "total_count": 3}, "IPR000922": {"hypothetical protein": 2, "total_count": 2}, "IPR004197": {"tail spike protein": 1, "Cytolethal distending toxin subunit B": 1, "hypothetical protein": 1, "total_count": 3}, "IPR015883": {"tail protein": 1, "Beta-N-acetylhexosaminidase": 1, "hypothetical protein": 1, "total_count": 3}, "IPR000852": {"N-succinylarginine dihydrolase": 1, "total_count": 1}, "IPR046372": {"hypothetical protein": 2, "total_count": 2}, "IPR007724": {"hypothetical protein": 2, "total_count": 2}, "IPR017736": {"6-phospho-beta-glucosidase BglB": 1, "hypothetical protein": 7, "Cryptic outer membrane porin BglH": 1, "integrase": 1, "total_count": 10}, "IPR015165": {"tail sheath": 1, "total_count": 1}, "IPR025092": {"hypothetical protein": 3, "dehydrogenase": 1, "total_count": 4}, "IPR027260": {"2OG-Fe(II) oxygenase": 1, "hypothetical protein": 2, "total_count": 3}, "IPR035394": {"hypothetical protein": 5, "total_count": 5}, "IPR001554": {"hypothetical protein": 3, "total_count": 3}, "IPR011613": {"hypothetical protein": 2, "total_count": 2}, "IPR000165": {"hypothetical protein": 2, "total_count": 2}, "IPR024746": {"hypothetical protein": 2, "glycosyltransferase": 1, "total_count": 3}, "IPR008811": {"tail spike protein": 1, "Alpha-xylosidase": 1, "total_count": 2}, "IPR027291": {"Poly-beta-1,6-N-acetyl-D-glucosamine N-deacetylase": 1, "total_count": 1}, "IPR044505": {"Glycogen debranching enzyme": 4, "hypothetical protein": 1, "Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 6}, "IPR040784": {"Glycogen debranching enzyme": 2, "hypothetical protein": 1, "total_count": 3}, "IPR010702": {"hypothetical protein": 1, "total_count": 1}, "IPR016288": {"hypothetical protein": 1, "total_count": 1}, "IPR036434": {"hypothetical protein": 1, "total_count": 1}, "IPR001137": {"UDP-N-acetyl-D-mannosaminuronic acid transferase": 1, "Ribonucleoside-diphosphate reductase subunit alpha": 1, "total_count": 2}, "IPR012480": {"hypothetical protein": 1, "total_count": 1}, "IPR004185": {"Carbohydrate acetyl esterase/feruloyl esterase": 1, "total_count": 1}}
import pprint
pp = pprint.PrettyPrinter(width = 150, compact = True)


In [2]:
pp.pprint(annotation)

{'IPR000125': {'hypothetical protein': 4, 'tail protein': 3, 'total_count': 7},
 'IPR000165': {'hypothetical protein': 2, 'total_count': 2},
 'IPR000322': {'Alpha-xylosidase': 1,
               'Isoprimeverose transporter': 1,
               'Sulfoquinovosidase': 2,
               'hypothetical protein': 1,
               'tail protein': 1,
               'total_count': 6},
 'IPR000400': {'baseplate hub subunit and tail lysozyme': 39,
               'baseplate protein': 1,
               'dehydrogenase': 1,
               'endolysin': 5,
               'hypothetical protein': 45,
               'total_count': 91},
 'IPR000490': {'Protein RecA': 12, 'UvsX-like recombinase': 92, 'total_count': 104},
 'IPR000514': {'6-phospho-beta-glucosidase BglB': 1,
               'hydrolase': 2,
               'hypothetical protein': 12,
               'major tail protein': 2,
               'minor tail protein': 1,
               'total_count': 20,
               'transcriptional regulator': 2},
 'IP

In [5]:
annot_diff = {}
for ipr in annotation :
    if ipr not in annotation_1 :
         annot_diff[ipr] = annotation[ipr]

In [6]:
pp.pprint(annot_diff)

{'IPR000322': {'Alpha-xylosidase': 1,
               'Isoprimeverose transporter': 1,
               'Sulfoquinovosidase': 2,
               'hypothetical protein': 1,
               'tail protein': 1,
               'total_count': 6},
 'IPR001223': {'Dda-like helicase': 1,
               'endolysin': 1,
               'hypothetical protein': 15,
               'major tail protein': 3,
               'minor tail protein': 66,
               'protease': 1,
               'tail protein': 5,
               'total_count': 92},
 'IPR006047': {'Glycogen debranching enzyme': 4,
               'Oligo-1,6-glucosidase': 2,
               'Periplasmic alpha-amylase': 1,
               'Tn3 family transposase TnAs1': 2,
               'Tn3 family transposase TnAs3': 1,
               'Trehalose-6-phosphate hydrolase': 3,
               'endolysin': 1,
               'head maturation protease': 24,
               'hydrolase': 1,
               'hypothetical protein': 8,
               'major tail p